### Fine-tuning Transformer models to generate summaries of hindi news articles
- The code in this notebook is based on [this](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization.ipynb#scrollTo=545PP3o8IrJV) tutorial

In [ ]:
! pip install datasets transformers rouge-score nltk sentencepiece

- Restart the runtime (once) after installing the packages

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# model_checkpoint = "t5-small"
# model_checkpoint = "facebook/mbart-large-cc25"     # Link: https://huggingface.co/docs/transformers/model_doc/mbart
model_checkpoint = "google/mt5-small"
# model_checkpoint = "facebook/mbart-large-50-many-to-many-mmt"

## Loading Custom Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/amankhullar/hindi_summarization.git

Cloning into 'hindi_summarization'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 22 (delta 4), reused 21 (delta 3), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [ ]:
import os
import sys
import torch

import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from transformers import MBartForConditionalGeneration, MBartTokenizer

In [ ]:
class HindiSumDataset(Dataset):
    def __init__(self, article_encodings, summary_encodings):
        self.article_encodings = article_encodings
        self.summary_encodings = summary_encodings

    def __len__(self):
        return len(self.article_encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.article_encodings.items()}
        item['labels'] = self.summary_encodings['input_ids']
        print(item)
        return item

In [ ]:
base_pth = '/content/drive/MyDrive/GaTech/NLP/hindi_summarization/'
train_pth = os.path.join(base_pth, 'archive-2', 'train.csv')
test_pth = os.path.join(base_pth, 'archive-2', 'test.csv')

# train_df = pd.read_csv(train_pth)
# test_df = pd.read_csv(test_pth)

### Dataset preprocessing

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, src_lang="hi_IN", tgt_lang="hi_IN")

# tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25", src_lang="hi_IN", tgt_lang="hi_IN")

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
def make_data_class(train_pth, test_pth):
    """
    This function has been replaced by a new method.
    """
    train_df = pd.read_csv(train_pth)
    test_df = pd.read_csv(test_pth)
    num_articles_to_train = 100 ### NOTE: change this during final experiment
    max_input_length = 1024
    max_target_length = 256

    train_articles, train_summaries = train_df['article'].fillna('').tolist()[:num_articles_to_train], train_df['headline'].fillna('').tolist()[:num_articles_to_train]
    test_articles, test_summaries = test_df['article'].fillna('').tolist()[:num_articles_to_train], test_df['headline'].fillna('').tolist()[:num_articles_to_train]

    if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b", "google/mt5-small"]:
        prefix = "summarize: "
        train_articles = [prefix + article for article in train_articles]
        test_articles = [prefix + article for article in test_articles]

    train_articles_encodings = tokenizer(train_articles, truncation=True, padding=True, max_length=max_input_length)
    with tokenizer.as_target_tokenizer():
        train_summaries_encodings = tokenizer(train_summaries, truncation=True, padding=True, max_length=max_target_length)

    test_articles_encodings = tokenizer(test_articles, truncation=True, padding=True, max_length=max_input_length)
    with tokenizer.as_target_tokenizer():
        test_summaries_encodings = tokenizer(test_summaries, truncation=True, padding=True, max_length=max_target_length)

    train_dataset = HindiSumDataset(train_articles_encodings, train_summaries_encodings)
    test_dataset = HindiSumDataset(test_articles_encodings, test_summaries_encodings)

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("hindi_summarization/hindisumdataset")

metric = load_metric("rouge")

Using custom data configuration default


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset hindi_sum downloaded and prepared to /root/.cache/huggingface/datasets/hindi_sum/default/1.2.0/e58e6a0aed0b801e10d0b53103001b4ce7f009ffa6225d5e356a54a37a8f77e8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

- 100,000 training articles
- 66,653 testing articles

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [24]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 49426
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 9987
    })
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
raw_datasets["train"][0]

{'document': 'दिल्ली में सुप्रीम कोर्ट के डीज़ल टैक्सियों को बंद करने के फैसले के बाद हजारों टैक्सी ड्राइवरों की रोजी रोटी पर तो असर पड़ा ही है, लेकिन अब दिल्ली पर एक और नई मुसीबत आ गई है. चुनाव आयोग राजधानी के 13 वार्ड में उपचुनाव करवा रहा है, लेकिन चुनावों से दो हफ्ते पहले चुनाव आयोग में कामकाज ठप्प हो गया है.\nकमीशन ने किराए पर ली थी डीजल गाड़ियां\n\nदरअसल कमीशन ने लगभग सौ गाड़ियां चुनाव के कामकाज को करने के लिए किराए पर लीं, जिनमें सभी\nडीज़ल से चलने वाली टैक्सी\nथी. इन्हीं टैक्सियों से चुनाव अधिकारी से लेकर चुनावों का जिम्मा संभालने वाले बाकी कर्मचारी भी एक जगह से दूसरी जगह आते जाते थे. अचानक चुनावों से ठीक पहले आई इस परेशानी ने दिल्ली चुनाव आयोग का कामकाज ही ठप्प कर दिया है.\n\nरियायत के लिए की जा सकती है मांग\n\nदिल्ली के राज्य चुनाव अधिकारी राकेश मेहता ने इस मुश्किल का रास्ता निकालने के लिए मंगलवार को दिल्ली के पुलिस कमिश्नर और ट्रांसपोर्ट कमिश्नर की बैठक बुलाई है. इस बैठक में राज्य चुनाव आयुक्त 15 मई को होने वाले चुनावों को लेकर\nगाड़ियों की उपलब्धता\nको लेकर पुलिस और सरकार से

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])
# show_random_elements(train_dataset)

,document,summary,id
0,"यूपी की राजधानी लखनऊ में बदमाशों ने एक वकील की गोली मारकर हत्या कर दी. वारदात के वक्त वकील अपनी कार में सवार था. बदमाशों ने उनको सीने में गोली मारी थी.\nबाइक सवार बदमाशों ने मारी गोली\n\nमामला राजधानी के पॉश इलाके विभूतिखण्ड का है. निशातगंज निवासी\nअधिवक्ता\nसंजय शर्मा बुधवार की देर शाम अपनी कार (यूपी 32जीबी 9798) से शहीद पथ के पास गए थे. इसी दौरान जब वह वहां पहुंचे तो अज्ञात बाइक सवार बदमाशों ने उन्हें रोक लिया और उनके सीने से पिस्टल सटाकर गोली मार दी.\n\nमौके पर पहुंची पुलिस\n\nवारदातो को अंजाम देकर बदमाश मौके से फरार हो गए. गोली की आवाज़ सुनकर आस-पास लोग घटना स्थल की तरफ भागे. और पुलिस को वारदात की सूचना दी. पुलिस ने मौके पर पहुंच कर संजय को लोहिया अस्पताल में भर्ती कराया, जहां से उसे ट्रॉमा सेंटर भेज दिया गया. डॉक्टरों ने वहां वकील को मृत घोषित कर दिया.\n\nदो लोग हिरासत में\n\nअधिवक्ता की हत्या से पुलिस विभाग सकते में आ गया. वारदात के बाद पूरे शहर की नाकेबंदी करके वाहनों की चैकिंग की गई. डीआईजी आर.के.एस. राठौर ने बताया कि पूरे प्रकरण की जांच की जा रही है. इस संबंध में शक के आधार पर दो लोगों को हिरासत में लिया गया है. पुलिस जल्द ही इस हत्या का खुलासा कर देगी.\nपहले भी हुई थी वकील की हत्या\n\nगौरतलब है कि पिछले साल 21 जनवरी 2015 को पीजीआई थाना क्षेत्र में बाइक सवार तीन बदमाशों ने वकील निखिलेंद्र कुमार पर बम से हमला किया था. जिसमें की उसकी मौत हो गई थी. इसके अलावा बीती 10 फरवरी 2016 को नाका थाना क्षेत्र बाराबंकी निवासी 36 वर्षीय श्रवण कुमार का शव एक मंदिर के पास खून से लथपथ मिला था.",लखनऊ में वकील की गोली मारकर हत्या,12829
1,"बंगलुरु टेस्ट में ऑस्ट्रेलिया को 75 रनों से हराने के साथ टीम इंडिया ने अपने नाम एक बड़ी उपलब्धि हासिल कर ली. भारत ने ऑस्ट्रेलिया को 188 रनों का टारगेट दिया था. लेकिन ऑस्ट्रेलिया 112 रनों पर ढेर हो गया. यानी 200 से कम के टारगेट का बचाव करते हुए सबसे बड़ी जीत की बात करें, तो टेस्ट क्रिकेट की यह तीसरी बड़ी जीत मानी जाएगी.\n200 से कम के टारगेट की रक्षा करते हुए अब तक की सबसे बड़ी जीत 1994 में वेस्टइंडीज ने पाई थी, जब उसने इंग्लैंड को 147 रनों से हराया था. देखिए ये लिस्ट-\n\n1. 1994, पोर्ट ऑफ स्पेन, 194 के टारगेट के आगे इंग्लैंड 46 पर ढेर, वेस्टइंडीज 147 रनों से ये टेस्ट जीता.\n\n2. 1911, मेलबर्न, 170 के टारगेट के आगे द. अफ्रीका 80 पर ढेर, ऑस्ट्रेलिया ने 89 रनों से ये टेस्ट जीता.\n\n3. 2017, बंगलुरु, 188 के टारगेट के आगे ऑस्ट्रेलिया 112 पर ढेर, भारत ने 75 रनों से ये टेस्ट जीता.\n\n- भारत ने अपने छोटे लक्ष्यों का कब- कब बचाव किया\n\n2004 में 107 के टारगेट के आगे ऑस्ट्रेलिया 93 रनों पर ढेर\n\n1981 में 143 के टारगेट के आगे ऑस्ट्रेलिया 83 रनों पर ढेर\n\n1996 में 170 के टारगेट के आगे द. अफ्रीका 105 रनों पर ढेर\n\n1969 में 188 के टारगेट के आगे न्यूजीलैंड 127 रनों पर ढेर\n\n2017 में 188 के टारगेट के आगे ऑस्ट्रेलिया 112 रनों पर ढेर\n\n\n\n\n\n\n\nपहली पारी में पिछड़ने के बाद भारत की घरेलू जीत\n\n274 v ऑस्ट्रेलिया, कोलकाता, 2001\n\n99 v ऑस्ट्रेलिया, मुंबई, 2004\n\n87 v ऑस्ट्रेलिया, बंगलुरु, 2017",200 से कम टारगेट पर टेस्ट क्रिकेट की तीसरी सबसे बड़ी जीत है ये,31678
2,टीम इंडिया के बल्‍लेबाज गौतम गंभीर पर एक टेस्‍ट के लिए लगाए गए प्रतिबंध को वीरेंद्र सहवाग ने ज्‍यादती करार दिया है. उनका कहना है कि गंभीर को दी गई सजा कुछ ज्‍यादा है.\n\n\nविस्‍फोटक बल्‍लेबाज वीरेंद्र सहवाग ने एक टेस्‍ट मैच के लिए प्रतिबंधित किए गए गौतम गंभीर का खुलकर बचाव किया है. सहवाग का मानना है कि गंभीर पर बैन लगाया जाना ज्‍यादती है. सहवाग ने कहा है कि गंभीर को मैच से मिली राशि में कटौती करके भी छोड़ा जा सकता था.\n\n\nदूसरी ओर बीसीसीआई इस बैन के खिलाफ आईसीसी से अपील कर चुकी है. अब आईसीसी को इस बारे में अंतिम फैसला करना है. हालांकि गौतम गंभीर ने एक साल के भीतर दूसरी बार इस तरह की गलती की है.\n\n\n\nगौर करने वाली बात यह है कि गंभीर के खिलाफ ऑस्‍ट्रेलिया ने औपचारिक रूप से शिकायत दर्ज नहीं करवाई है. इसके बावजूद मैच रेफरी क्रिस ब्रॉड ने उन्‍हें एक मैच के लिए प्रतिबंधित करने का निर्णय किया. क्रिकेट के पंडितों का मानना है कि क्रिस ब्रॉड पहले भी भारतीय खिलाडि़यों के खिलाफ पूर्वाग्रह से प्रेरित फैसले देते रहे हैं.,गौतम गंभीर पर बैन ज्‍यादती: वीरेंद्र सहवाग,29908
3,"महाराष्ट्र के चंद्रपुर में तीन बाघों की संदिग्ध परिस्थिति में मौत हो गई है. इनमें एक बाघिन और उसके दो शावक शामिल हैं. मौके पर पहुंचे वन विभाग के अधिकारी जांच में जुट गए हैं. द

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

## Preprocessing the data

If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b", "google/mt5-small"]:
    # prefix = "summarize: "
    prefix = "इस वाक्य को सारांशित करें: "
    print("here")
else:
    prefix = ""

here


In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    # inputs = [prefix + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)
        # labels = tokenizer(examples['headline'], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer, TrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
BATCH_SIZE = 4
NUM_EPOCHS = 3

model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-mbart",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    fp16=False,             ## Changed here as well
    push_to_hub=False,      ## Change here to push to hub
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # print("Predicted: {}".format(decoded_preds))
    # print("Ground truth: {}".format(decoded_labels))
    with open('predicted.txt', 'w') as f:
        f.write('\n'.join(decoded_preds))
    with open('ground_truth.txt', 'w') as f:
        f.write('\n'.join(decoded_labels))
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
print(model)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (w

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
from transformers import Trainer
trainer = Seq2SeqTrainer( 
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    # train_dataset=train_dataset,
    eval_dataset=tokenized_datasets["validation"],
    # eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
type(tokenized_datasets["validation"])

datasets.arrow_dataset.Dataset

In [ ]:
# len(test_dataset)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: id, summary, document. If id, summary, document are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 49426
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 37071


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.425700,1.899691,7.909500,1.301900,7.846500,7.857900,17.930000
2,2.226100,1.788773,8.543800,1.406000,8.499300,8.505600,18.314200
3,2.142600,1.758303,8.804100,1.500200,8.770100,8.788400,18.426000


Saving model checkpoint to mt5-small-finetuned-mbart/checkpoint-500
Configuration saved in mt5-small-finetuned-mbart/checkpoint-500/config.json
Model weights saved in mt5-small-finetuned-mbart/checkpoint-500/pytorch_model.bin
tokenizer config file saved in mt5-small-finetuned-mbart/checkpoint-500/tokenizer_config.json
Special tokens file saved in mt5-small-finetuned-mbart/checkpoint-500/special_tokens_map.json
Copy vocab file to mt5-small-finetuned-mbart/checkpoint-500/spiece.model
Saving model checkpoint to mt5-small-finetuned-mbart/checkpoint-1000
Configuration saved in mt5-small-finetuned-mbart/checkpoint-1000/config.json
Model weights saved in mt5-small-finetuned-mbart/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in mt5-small-finetuned-mbart/checkpoint-1000/tokenizer_config.json
Special tokens file saved in mt5-small-finetuned-mbart/checkpoint-1000/special_tokens_map.json
Copy vocab file to mt5-small-finetuned-mbart/checkpoint-1000/spiece.model
Saving model checkpo

TrainOutput(global_step=37071, training_loss=2.5086690922209978, metrics={'train_runtime': 12010.6503, 'train_samples_per_second': 12.346, 'train_steps_per_second': 3.087, 'total_flos': 8.36359944148992e+16, 'train_loss': 2.5086690922209978, 'epoch': 3.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```